In [1]:
import faiss
import numpy as np
from utils import read_jsonl, fetch_with_retry
from sentence_transformers import SentenceTransformer

In [2]:
data = read_jsonl("../autodl-tmp/data3_2.jsonl")

In [10]:
class VectorDatabase:
    def __init__(self, dimension):
        """初始化 FAISS 索引"""
        self.dimension = dimension
        self.model = SentenceTransformer("../all-mpnet-base-v2", local_files_only=True)
        self.index = faiss.IndexFlatL2(dimension)  # 使用 L2 距离索引
        self.texts = []  # 存储文本信息

    def add_embeddings(self, embeddings, texts):
        """添加已有的嵌入向量到 FAISS 索引"""
        embeddings = np.array(embeddings, dtype=np.float32)
        self.index.add(embeddings)
        self.texts.extend(texts)
        print(f"已添加 {len(texts)} 条数据到 FAISS 索引。")

    def search(self, query_embedding, top_k=5):
        """基于嵌入向量进行相似度搜索"""
        query_embedding = np.array([query_embedding], dtype=np.float32)
        distances, indices = self.index.search(query_embedding, top_k)
        results = [(self.texts[i], distances[0][j]) for j, i in enumerate(indices[0]) if i != -1]
        return results

    def generate_prompt(self, query, retrieved_texts):
        """基于查询与检索的文本，构造提示词"""
        prompt_template = f"""
        你是一名舆情分析专家。请根据以下用户输入的查询，结合相关的舆情信息进行回答，你的主要任务是分析一下该事件是否可能演变为负面舆情事件。

        **用户查询**：
        {query}

        **相关舆情信息**：
        {retrieved_texts}

        请用简洁准确的语言给出你的分析：
        """
        return prompt_template

In [11]:
dimension = len(data[0]["embeddings"])

vector_db = VectorDatabase(dimension)

# 添加已有的 embeddings 和对应文本
embeddings = [item["embeddings"] for item in data]
texts = [item["text"] for item in data]
vector_db.add_embeddings(embeddings, texts)

已添加 8983 条数据到 FAISS 索引。


In [12]:
query = "抖音直播间有什么套路吗"
query_embedding = vector_db.model.encode(query, convert_to_numpy=True)  # 假设查询语句的 embedding
results = vector_db.search(query_embedding, top_k=10)

for text, score in results:
    print(f"相关文本: {text} (相似度: {score})")

相关文本: 总台记者发现，坠机现场仍有燃油味 (相似度: 0.08274109661579132)
相关文本: 冲突因检察官有很多倾向性讯问激怒了被告人 (相似度: 0.2400105595588684)
相关文本: 物业称疑有人恶意毒狗，己报警 (相似度: 0.25180700421333313)
相关文本: 盗墓有风险，铁窗两行泪 (相似度: 0.3277413845062256)
相关文本: 滑倒有风险，疼完两行泪 (相似度: 0.3277413845062256)
相关文本: 如果仅仅是完全随机的选择，有可能导致算法收敛很慢 (相似度: 0.3548785150051117)
相关文本: 如何看待“炫富挑战”爆红现象“炫富”，在百度百科有这样一段解释：“指炫耀、夸耀财富或富有 (相似度: 0.3656255006790161)
相关文本: 因附近没有可燃物，所以没有引发火灾 (相似度: 0.3859977126121521)
相关文本: 有研究发现，推特上的舆论发展并稳定形成主流意见的速度非常快，这对于那些能够在早期左右舆论的大型组织非常有利 (相似度: 0.39239734411239624)
相关文本: 当前，整体舆情热度有所回落 (相似度: 0.39244943857192993)


In [13]:
retrieved_texts = "\n".join([text for text, score in results])

In [14]:
final_prompt = vector_db.generate_prompt(query, retrieved_texts)

In [15]:
response = fetch_with_retry(final_prompt)

In [16]:
print(response.choices[0].message.content)

根据现有信息分析，抖音直播间“套路”问题存在演变为负面舆情的潜在风险，但当前暂无直接关联的舆情发酵迹象。判断依据如下：  
1. **风险关联性**：直播间常见的诱导消费、虚假宣传等套路天然具备负面舆情属性，与用户财产安全、平台信任度直接相关，敏感度高；  
2. **舆情现状**：提供的舆情信息中未提及抖音直播相关负面事件，且整体舆情热度回落，显示短期内暂无叠加风险；  
3. **触发条件**：若出现典型案例（如大规模用户投诉、头部主播被曝欺诈），或媒体/KOL集中曝光套路操作手法，可能快速引发舆论扩散。  

**结论**：当前暂未形成实际负面舆情，但需警惕该话题的敏感性，建议持续监测直播间消费纠纷类新增信息。
